# Modeling notebook

1. This notebook implements 3 models
    *   A lineal regression model as baseline
    *   A tree-based model as potential candidate and also as a pipeline to do Feature importance
    *   A MLP to use as the main regression model

### Training and test datasets preparation

In [244]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

class FeatureEngineering(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.centroids = None


    def __compute_distance(self, point, centroid):
        return np.sqrt((point[0] - centroid[0])**2 + (point[1] - centroid[1])**2)

    def __distance_to_closest_centroid(self, row):
        distances = [self.__compute_distance([row['latitude'], row['longitude']], centroid) for centroid in self.centroids]
        return min(distances)

    def __get_distance_to_high_value_area(self, X):
        if self.centroids is None:
            raise ValueError('Fit the model first before transforming the data')
        
        X['distance_to_high_value_area'] = X.apply(lambda row: self.__distance_to_closest_centroid(row), axis=1)
        return X

    # Fit the KMeans clustering model for the distance to high value area feature
    def fit(self, X, y=None, num_clusters=2):
        threshold = X['median_house_value'].quantile(0.80)
        high_value_homes = X[X['median_house_value'] > threshold]

        kmeans = KMeans(n_clusters=num_clusters, random_state=10)
        high_value_homes['cluster'] = kmeans.fit_predict(high_value_homes[['latitude', 'longitude']])
        self.centroids = kmeans.cluster_centers_

        return self
    
    def transform(self, X):
        X['bedrooms_per_room'] = X['total_bedrooms'] / X['total_rooms']
        X['population_per_household'] = X['population'] / X['households']
        X['rooms_per_household'] = X['total_rooms'] / X['households']
        X['income_per_household'] = X['median_income'] / X['households']
        X['income_per_bedroom'] = X['median_income'] / X['total_bedrooms']
        X['income_per_room'] = X['median_income'] / X['total_rooms']
        X['age_of_population'] = X['housing_median_age'] / X['population']
        X['age_of_households'] = X['housing_median_age'] / X['households']
        X['population_density'] = X['population'] / X['total_rooms']
        X['bedrooms_per_population'] = X['total_bedrooms'] / X['population']
        X['is_capped'] = X['median_house_value'] == 500000
        

        self.__get_distance_to_high_value_area(X)
        
        return X

# Custom transformer to drop a column    
class DropColumnTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if self.column_name in X.columns:
            X = X.drop(columns=[self.column_name])
        return X
    
housing_data = pd.read_csv("../data/housing.csv")
train_set, test_set = train_test_split(housing_data, test_size=0.3, random_state=10)

# Keep the original labels for training and testing
y_train = train_set['median_house_value'].to_numpy()
y_test = test_set['median_house_value'].to_numpy()

num_attribs = train_set.columns.tolist()
num_attribs.remove('ocean_proximity')
cat_attribs = ['ocean_proximity']

feature_eng = FeatureEngineering()
feature_eng.fit(train_set)

num_pipeline = Pipeline([
    ('feature_eng', feature_eng),
    ('drop_column', DropColumnTransformer('median_house_value')),
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler()),
])

pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs),
])

X_train = pipeline.fit_transform(train_set)
X_test = pipeline.transform(test_set)

print(X_train.shape)
print(X_test.shape)

/tmp/ipykernel_1075/584058149.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_value_homes['cluster'] = kmeans.fit_predict(high_value_homes[['latitude', 'longitude']])
/tmp/ipykernel_1075/584058149.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_value_homes['cluster'] = kmeans.fit_predict(high_value_homes[['latitude', 'longitude']])


(14448, 25)
(6192, 25)


### Train and test a linear regression model

To evaluate:
1. The Root Mean Squared Error is used for the training error 
2. N-fold cross validation with RMSE is used, with 10 folds for the validation error

In [245]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
lin_predictions = lin_reg.predict(X_train)

lin_rmse = np.sqrt(mean_squared_error(y_train, lin_predictions))
lin_scores = cross_val_score(lin_reg, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)

print('Linear Regression RMSE:', lin_rmse)
print('Linear Regression RMSE Scores:', lin_rmse_scores)
print('Linear Regression RMSE Mean:', lin_rmse_scores.mean())
print('Linear Regression RMSE Standard Deviation:', lin_rmse_scores.std())    


Linear Regression RMSE: 65512.19687815971
Linear Regression RMSE Scores: [65491.56395045 62939.19496873 64423.00782818 65956.13642558
 67187.4882399  66054.65590843 64872.79467416 79491.74134393
 65726.6694338  70087.43228025]
Linear Regression RMSE Mean: 67223.06850533944
Linear Regression RMSE Standard Deviation: 4457.011346877264


To note:

1. It seems that the model is underfitting as the error is around $67k
2. We could try to:
    * Use a more powerful model
    * Add more features that give more signal

### Training a decision tree model

In [246]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train, y_train)
tree_pred = tree_reg.predict(X_train)

tree_rmse = np.sqrt(mean_squared_error(y_train, tree_pred))
scores = cross_val_score(tree_reg, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
tree_rmse_scores = np.sqrt(-scores)

print(f"RMSE: {tree_rmse}")
print(f"Scores: {tree_rmse_scores}")
print(f"Mean: {tree_rmse_scores.mean()}")
print(f"Standard Deviation: {tree_rmse_scores.std()}")


RMSE: 0.0
Scores: [66295.36308594 60796.25593123 68176.65528911 66208.60843192
 70429.140568   69390.16185295 68109.15537166 65748.78157228
 66863.53809333 64677.67522789]
Mean: 66669.5335424307
Standard Deviation: 2560.8818256409218


To note:

1. Decision tree regressor is overfitting the data (Training RMSE = 0)
2. Performance is a little bit better than the lineal regressor model when cross-validated

### Training a random forest model

In [247]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

forest_reg = RandomForestRegressor()
forest_reg.fit(X_train, y_train)
forest_pred = forest_reg.predict(X_train)

forest_rmse = np.sqrt(mean_squared_error(y_train, forest_pred))
scores = cross_val_score(forest_reg, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
forest_rmse_scores = np.sqrt(-scores)

print(f"RMSE: {forest_rmse}")
print(f"Scores: {forest_rmse_scores}")
print(f"Mean: {forest_rmse_scores.mean()}")
print(f"Standard Deviation: {forest_rmse_scores.std()}")


RMSE: 17635.61412999737
Scores: [46490.76984445 44447.25210052 47443.05010518 45810.53875402
 48035.16763856 49020.16403656 46447.96852739 49081.04351026
 46749.26444453 49525.37010026]
Mean: 47305.05890617198
Standard Deviation: 1538.98724340286


To note:

1. The model stills overfit a lot but given that random forest is an ensemble mechanism, this helps to generalize and regularize so the overfitting is not as bad as the decision tree
2. We could limit the 'max_depth', 'min_samples_split' and 'min_samples_leaf' to force the model to limit the complexity of the tree

### Multi Layered Perceptron (MLP)

In [248]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import cross_val_score


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'{device=}')

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.5):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.bn2 = nn.BatchNorm1d(hidden_size // 2)
        self.fc3 = nn.Linear(hidden_size // 2, hidden_size // 4)
        self.bn3 = nn.BatchNorm1d(hidden_size // 4)
        self.fc4 = nn.Linear(hidden_size // 4, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

# Custom wrapper class to use PyTorch model with scikit-learn cross-validation
class PyTorchRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, model, epochs=100, lr=0.01):
        self.model = model.to(device)
        self.epochs = epochs
        self.lr = lr

    def fit(self, X, y):
        X = torch.tensor(X, dtype=torch.float32).to(device)
        y = torch.tensor(y, dtype=torch.float32).view(-1, 1).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        
        self.model.train()
        for _ in range(self.epochs):
            optimizer.zero_grad()
            outputs = self.model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
        return self

    def predict(self, X):
        X = torch.tensor(X, dtype=torch.float32).to(device)
        self.model.eval()
        with torch.no_grad():
            predictions = self.model(X).cpu().numpy()
        return predictions.flatten()



device=device(type='cuda')


In [249]:
# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 500
output_size = 1
learning_rate = 0.1
dropout_rate = 0.3
epochs = 2000

mlp = MLP(input_size, hidden_size, output_size, dropout_rate=dropout_rate)
mlp_regressor = PyTorchRegressor(model=mlp, epochs=epochs, lr=learning_rate) 
mlp_regressor.fit(X_train, y_train)
mlp_predictions = mlp_regressor.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, mlp_predictions))
scores = cross_val_score(mlp_regressor, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
rmse_scores = np.sqrt(-scores)

print(f"RMSE: {rmse}")
print(f"Scores: {rmse_scores}")
print(f"Mean: {rmse_scores.mean()}")
print(f"Standard Deviation: {rmse_scores.std()}")


RMSE: 28235.45978784756
Scores: [41286.50774169 39449.54959096 40942.04614358 44028.68295994
 42819.39256196 43851.85429964 44263.38193735 44749.68769051
 41582.43147122 44579.65542744]
Mean: 42755.318982427954
Standard Deviation: 1736.5787071620084


To note:

1. The MLP has slightly worse performance to the Random Forest regressor but the overfitting is not as bad as Random Forest.
2. On GPU, the training time is the same as Random Forest on CPU, so this is a constrain.
3. Hyperparameter optimization of this model was done manually

## Hyperparamter optimization and Feature importance

1. Do hyperparameter optimization for the Random Forest regressor
2. Do Feature Importance to get an idea of the performance of the features

In [250]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Create the grid search object
grid_search = GridSearchCV(estimator=forest_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = np.sqrt(-grid_search.best_score_)

print('Best Parameters:', best_params)
print('Best RMSE Score:', best_score)


Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 30}
Best RMSE Score: 50057.95987145539


### Testing RF with best parameters

In [251]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import numpy as np

forest_reg = RandomForestRegressor(
    max_depth=None,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=30
)

forest_reg.fit(X_train, y_train)

forest_pred = forest_reg.predict(X_train)

forest_rmse = np.sqrt(mean_squared_error(y_train, forest_pred))

scores = cross_val_score(forest_reg, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
forest_rmse_scores = np.sqrt(-scores)

print(f"RMSE: {forest_rmse}")
print(f"Scores: {forest_rmse_scores}")
print(f"Mean: {forest_rmse_scores.mean()}")
print(f"Standard Deviation: {forest_rmse_scores.std()}")

RMSE: 23514.40323603561
Scores: [49932.96191154 47813.1644265  47700.73229024 49190.93573037
 50845.7240867  51063.23020631 49971.94386957 50832.2712312
 48926.01813995 52710.69831983]
Mean: 49898.76802122211
Standard Deviation: 1471.93663100848


To note:

1. After doing grid search on the forest regressor, the best loss is around 50k, slightly better than the MLP

In [252]:
extra_attributes = ['bedrooms_per_room', 
                    'population_per_household', 
                    'rooms_per_household', 
                    'income_per_household', 
                    'income_per_bedroom', 
                    'income_per_room', 
                    'age_of_population', 
                    'age_of_households', 
                    'population_density', 
                    'bedrooms_per_population', 
                    'is_capped', 
                    'distance_to_high_value_area']

feature_importances = grid_search.best_estimator_.feature_importances_
cat_encoder = pipeline.named_transformers_['cat']
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attributes + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)


[(0.22812541578398277, 'median_income'),
 (0.11997758166974434, 'is_capped'),
 (0.10233567499491797, '<1H OCEAN'),
 (0.08133539991799771, 'age_of_households'),
 (0.054933949238275395, 'bedrooms_per_room'),
 (0.05354800585999428, 'population_per_household'),
 (0.050289366357560636, 'median_house_value'),
 (0.04585602004022396, 'longitude'),
 (0.04484695624952824, 'latitude'),
 (0.03749366274541716, 'population_density'),
 (0.0255555591421462, 'housing_median_age'),
 (0.021091486603471384, 'income_per_household'),
 (0.016056431348457394, 'income_per_room'),
 (0.015695218443333395, 'total_rooms'),
 (0.01509032965809341, 'rooms_per_household'),
 (0.014204469161580798, 'households'),
 (0.013827760406509734, 'income_per_bedroom'),
 (0.013417338286805466, 'age_of_population'),
 (0.011742629132884217, 'total_bedrooms'),
 (0.011529482925630325, 'distance_to_high_value_area'),
 (0.011164991205205428, 'population'),
 (0.008749217347609816, 'NEAR BAY'),
 (0.002191787030531861, 'ISLAND'),
 (0.00090

To note:

1. distance_to_high_value_area is not a very important feature, and to get this feature a clustering model has to be trained, so this feature is dropped. The hypothesis is that '<1H OCEAN>' is highly correlated with distance_to_high_value_area and there is redundancy in these 2 features
2. A threshold of 0.01 is going to be use to filter out the least important features

### Testing the MLP 

* New feature engineering pipeline to drop features and clustering functionalities

In [253]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

class FeatureEngineering(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X['bedrooms_per_room'] = X['total_bedrooms'] / X['total_rooms']
        X['is_capped'] = X['median_house_value'] == 500000
        X['population_per_household'] = X['population'] / X['households']
        X['rooms_per_household'] = X['total_rooms'] / X['households']
        X['income_per_household'] = X['median_income'] / X['households']
        X['income_per_bedroom'] = X['median_income'] / X['total_bedrooms']
        X['income_per_room'] = X['median_income'] / X['total_rooms']
        X['age_of_population'] = X['housing_median_age'] / X['population']
        X['age_of_households'] = X['housing_median_age'] / X['households']
        X['population_density'] = X['population'] / X['total_rooms']

        # Commented out as feature importance is low
        # X['bedrooms_per_population'] = X['total_bedrooms'] / X['population']
        # self.__get_distance_to_high_value_area(X) 
        
        return X

# Custom transformer to drop a column    
class DropColumnTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if type(X) == pd.DataFrame:
            if self.column in X.columns:
                X = X.drop(columns=[self.column])
        elif type(X) == np.ndarray:
            X = np.delete(X, self.column, axis=1)
        return X
    
housing_data = pd.read_csv("../data/housing.csv")
train_set, test_set = train_test_split(housing_data, test_size=0.3, random_state=10)

y_train = train_set['median_house_value'].to_numpy()
y_test = test_set['median_house_value'].to_numpy()

num_attribs = train_set.columns.tolist()
num_attribs.remove('ocean_proximity')
cat_attribs = ['ocean_proximity']

num_pipeline = Pipeline([
    ('feature_eng', FeatureEngineering()),
    ('drop_column', DropColumnTransformer('median_house_value')),
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('onehot', OneHotEncoder()),
])

pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs),
])

X_train = pipeline.fit_transform(train_set)
X_test = pipeline.transform(test_set)

drop_column = DropColumnTransformer([19,21,22]) # Drop INLAND, ISLAND, NEAR BAY
X_train = drop_column.transform(X_train)
X_test = drop_column.transform(X_test)

print(X_train.shape)
print(X_test.shape)

(14448, 20)
(6192, 20)


In [254]:
# Tuned hyperparameters
input_size = X_train.shape[1]
hidden_size = 800
output_size = 1
learning_rate = 0.1
dropout_rate = 0.5
epochs = 600

mlp = MLP(input_size, hidden_size, output_size, dropout_rate=dropout_rate)
mlp_regressor = PyTorchRegressor(model=mlp, epochs=epochs, lr=learning_rate) 
mlp_regressor.fit(X_train, y_train)

mlp_predictions = mlp_regressor.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, mlp_predictions))
scores = cross_val_score(mlp_regressor, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
rmse_scores = np.sqrt(-scores)

print("------ Training Dataset ------")
print(f"RMSE: {rmse}")
print(f"Scores: {rmse_scores}")
print(f"Mean: {rmse_scores.mean()}")
print(f"Standard Deviation: {rmse_scores.std()}")

------ Training Dataset ------
RMSE: 43322.5302452567
Scores: [47825.38820605 45533.56754512 46574.76817572 47895.53130858
 49635.92041949 49892.30977543 46562.36423438 51313.55822219
 47513.7041949  50130.04620175]
Mean: 48287.715828362496
Standard Deviation: 1772.1537980024962


To note:
* The feature engineering pipeline was reworked to remove the least important features and the performace improved a little bit
* Hyperparameter optimization to the MLP was done.

In [255]:
forest_reg = RandomForestRegressor(
    max_depth=None,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=30
)

forest_reg.fit(X_train, y_train)

RandomForestRegressor(max_features='sqrt', min_samples_split=5, n_estimators=30)

### Testing the MLP and RF with the test set

In [256]:
# Test dataset 
print("------ Testing Dataset ------")
mlp_predictions = mlp_regressor.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, mlp_predictions))
print(f"MLP RMSE: {rmse}")


forest_predictions = forest_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, forest_predictions))
print(f"RF RMSE: {rmse}")


------ Testing Dataset ------
MLP RMSE: 51335.27222143574
RF RMSE: 54955.7921407399


## Conclusions

1. The MLP will be used for the e2e system. The reasons are:
    * Better performance compared to RF optimized through Grid Search
    * Scaling a NN is more straight forward (Data scaling, Model parallelism/sharding, distributed training/inference, quamtization, etc)
    * As more data is available, the NN architecture can be arbitrarily more complex to fit the data.
2. The MLP is slightly overfitting the training dataset because the test set loss is a little bit bigger. However the discrepancy is small and it is acceptable 